# **llama2-7b-chat-hf chat with pdf:**

Played with the llama2-7b-chat-hf fine-tuned model to chat with our own pdf data. Here llama2-7b-chat-hf model is acessed through a huggingface repo. Simple UI is also created. We can use our own pdf data and continue a chat about the information in our pdf files through this amaizing model by MetaAI.


---



---



---



# 01.Install Required Libraries

**pypdf**
---
---
pypdf is a free and open-source pure-python PDF library capable of splitting, merging, cropping, and transforming the pages of PDF files.

**python-dotenv**
---
---
Python-dotenv is a useful library that allows you to manage environment variables in your Python applications. This is particularly helpful for managing app settings during development and maintaining secure, organized configurations.

**transformers**
---
---
Transformers (formerly known as pytorch-transformers and pytorch-pretrained-bert) provides state-of-the-art general-purpose architectures (BERT, GPT-2, RoBERTa, XLM, DistilBert, XLNet, CTRL...) for Natural Language Understanding (NLU) and Natural Language Generation (NLG) with over 32+ pretrained models in 100+ languages and deep interoperability between TensorFlow 2.0 and PyTorch.

**einops**
---
---
einops mitigates a number of issues common to the conventional tensor manipulation routines, represents a number of commonly used functions with a small API, and makes code more readable and reliable. We implement notation in a Python package that provides identical API across a variety of tensor frameworks.

**accelerate**
---
---
Accelerate provides an easy API to make your scripts run with mixed precision and on any kind of distributed setting (multi-GPUs, TPUs etc.) while still letting you write your own training loop. The same code can then runs seamlessly on your local machine for debugging or your training environment.

**langchain**
---
---
LangChain is a framework designed to simplify the creation of applications using large language models. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.

**bitandbytes**
---
---
The bitsandbytes is a lightweight wrapper around CUDA custom functions, in particular 8-bit optimizers, matrix multiplication (LLM. int8()), and quantization functions.

**sentence_transformers**
---
---
SentenceTransformers is a Python framework for state-of-the-art sentence, text and image embeddings. The initial work is described in our paper Sentence-BERT: Sentence Embeddings using Siamese BERT-Networks. You can use this framework to compute sentence / text embeddings for more than 100 languages.

**llama-index**
---
---
LlamaIndex is a "data framework" to help you build LLM apps. It provides the following tools: Offers data connectors to ingest your existing data sources and data formats (APIs, PDFs, docs, SQL, etc.)

In [ ]:
!pip install pypdf
!pip install python-dotenv
!pip install -q transformers einops accelerate langchain bitsandbytes
!pip install sentence_transformers
!pip install llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.6/272.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.7 MB/s eta 0:00:00


# 02.Import Required Libraries

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms import HuggingFaceLLM

# 03.Set the Document Data Path

In [ ]:
documents = SimpleDirectoryReader("/content/drive/MyDrive/projectLLM/llm-chat-with-pdf/data").load_data()

# 04.Set the System Prompt

In [ ]:
from llama_index.prompts.prompts import SimpleInputPrompt


system_prompt = "You are a Q&A assistant. Your goal is to answer questions as accurately as possible based on the instructions and context provided."



# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

# 05.Login to Huggingface CLI

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' cre

# 06.Download the Model

In [ ]:
import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# 07.Create Embeddings

In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LangchainEmbedding, ServiceContext

embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
)


# 08.Create Servive Context

In [ ]:
service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# 09.Load Document Data

In [ ]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

# 10.Make the Query and Print Response

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("What Subsidiaries of Apple Inc?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [ ]:
print(response)

Based on the provided context information, the subsidiaries of Apple Inc. are:
* Apple Asia Limited Hong Kong
* Apple Asia LLC Delaware, U.S.
* Apple Canada Inc. Canada
* Apple Computer Trading (Shanghai) Co., Ltd. China
* Apple Distribution International Limited Ireland
* Apple India Private Limited India
* Apple Insurance Company, Inc. Arizona, U.S.
* Apple Japan, Inc. Japan
* Apple Korea Limited South Korea
* Apple Operations Europe Limited Ireland
* Apple Operations International Limited Ireland
* Apple Operations Limited Ireland
* Apple Operations Mexico, S.A. de C.V. Mexico
* Apple Pty Limited Australia
* Apple Sales International Limited Ireland
* Apple South Asia (Thailand) Limited Thailand
* Apple Vietnam Limited Liability Company Vietnam

Note: The above list includes the subsidiaries mentioned in the context information. There may be other subsidiaries of Apple Inc. that are not mentioned in the provided information.


# 11.Create a Simple Chat UI

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML, display_html
import time
import warnings

# Suppress user warnings globally
warnings.filterwarnings("ignore", category=UserWarning)

# Create a chat output widget
chat_output = widgets.Output(
    layout=widgets.Layout(width='1200px', height='350px', overflow_y='scroll'),
    disabled=True,
)

# Create a chat input text box
chat_input = widgets.Text(
    placeholder='Type your message here...',
    layout=widgets.Layout(width='1200px'),
)

# Create a button to send messages with custom styling
send_button = widgets.Button(
    description='Send',
    layout=widgets.Layout(width='100px', height='40px', margin='0 auto', float='right'),
    style={'button_color': '#e75480'}
)

# Create custom CSS for styling
custom_css = HTML("""
<style>
#chat_output {
    background-color: lightgray;
    padding: 10px;
    margin-bottom: 10px;
    border-radius: 5px;
}

#chat_input {
    background-color: lightyellow;
    border: 2px solid lightblue;
    border-radius: 5px;
}

#send_button {
    background-color: #e75480 !important;
    color: black !important;
    border: none;
    border-radius: 5px;
    cursor: pointer;
}

#send_button:hover {
    background-color: hotpink !important;
}

#chatbot_heading {
    text-align: center;
    color: #e75480;
    font-size: 24px;
}
</style>
""")
display_html(custom_css)

# Create a function to handle button click
def send_message(b):
    user_message = chat_input.value.strip()
    if user_message:
        with chat_output:
            print(f'User: {user_message}')
        chat_input.value = ''  # Clear the input box
        time.sleep(2)  # Simulate a delay (replace with actual bot response logic)
        with chat_output:
            response = query_engine.query(user_message)
            print('Llama2 Bot:', response)  # Replace with your bot's response


# Attach the send_message function to the button click event
send_button.on_click(send_message)

# Display the chat UI with a pink heading
display(HTML("<h1 id='chatbot_heading'>Simple Chatbot</h1>"))
display(chat_output)
display(chat_input)
display(send_button)

Output(layout=Layout(height='350px', overflow_y='scroll', width='1200px'))

Text(value='', layout=Layout(width='1200px'), placeholder='Type your message here...')

Button(description='Send', layout=Layout(height='40px', margin='0 auto', width='100px'), style=ButtonStyle(but…